In [1]:
import cv2, numpy as np
import math
import time
import random
from matplotlib import pyplot as plt
%matplotlib inline
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, push_notebook
import argparse
import os.path

output_notebook()

Loading BokehJS ...

# YOLO 관련

In [2]:
# 초기값 설정
confThreshold = 0.5  # Confidence threshold
nmsThreshold = 0.4   # Non-maximum suppression threshold
inpWidth = 416       # Width of network's input image
inpHeight = 416      # Height of network's input image

# yolo 네트워크 불러오기
classesFile = "coco.names"
classes = None
with open(classesFile, 'rt') as f : classes = f.read().rstrip('\n').split('\n')

modelConfiguration = "yolov3.cfg"
modelWeights = "yolov3.weights"

net = cv2.dnn.readNetFromDarknet(modelConfiguration, modelWeights)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [3]:
# 출력 레이어 가져오기
def getOutputsNames(net):
    layersNames = net.getLayerNames()
    return [layersNames[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# bounding box 그리기
def drawPred(classId, conf, left, top, right, bottom):
    cv2.rectangle(frame, (left, top), (right, bottom), (255, 178, 50), 3)
    
    label = '%.2f' % conf
        
    if classes:
        assert(classId < len(classes))
        label = '%s:%s' % (classes[classId], label)

    labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    top = max(top, labelSize[1])
    cv2.rectangle(frame, (left, top - round(1.5*labelSize[1])), (left + round(1.5*labelSize[0]), top + baseLine),
                 (255, 255, 255), cv2.FILLED)
    cv2.putText(frame, label, (left, top), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,0,0), 2)

# NMS를 적용하여 낮은 confidence의 bounding box 제거
def postprocess(frame, outs):
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        drawPred(classIds[i], confidences[i], left, top, left + width, top + height)

### yolo 영상 저장

In [11]:
cap = cv2.VideoCapture('C:/Users/i/Downloads/final project/lanenet-lane-detection-master/test4.mp4')

# 프레임 너비/높이, 초당 프레임 수 확인
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH) # 또는 cap.get(3)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT) # 또는 cap.get(4)
fps = cap.get(cv2.CAP_PROP_FPS) # 또는 cap.get(5)
print('프레임 너비: %d, 프레임 높이: %d, 초당 프레임 수: %d' %(width, height, fps))

fourcc = cv2.VideoWriter_fourcc(*'DIVX') # 코덱 정의
out = cv2.VideoWriter('output.mp4', fourcc, fps, (int(width), int(height))) # VideoWriter 객체 정의

프레임 너비: 904, 프레임 높이: 512, 초당 프레임 수: 29


In [6]:
while cap.isOpened() : 
    ret, frame = cap.read()

    if not ret:
        print("프레임을 수신할 수 없습니다(스트림 끝?). 종료 중 ...")
        break

    blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)

    net.setInput(blob)
    outs = net.forward(getOutputsNames(net))
    postprocess(frame, outs)
    
    out.write(frame)
    cv2.imshow('out', frame)
    if cv2.waitKey(40) == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

프레임을 수신할 수 없습니다(스트림 끝?). 종료 중 ...


### detection results 좌표 구하기

In [30]:
cap = cv2.VideoCapture('C:/Users/i/Downloads/final project/lanenet-lane-detection-master/test4.mp4')
currentFrame = 0
while cap.isOpened() : 
    ret, frame = cap.read()

    if not ret:
        print("프레임을 수신할 수 없습니다(스트림 끝?). 종료 중 ...")
        break

    blob = cv2.dnn.blobFromImage(frame, 1/255, (inpWidth, inpHeight), [0,0,0], 1, crop=False)
    
    net.setInput(blob)
    outs = net.forward(getOutputsNames(net))
    
    frameHeight = frame.shape[0]
    frameWidth = frame.shape[1]

    classIds = []
    confidences = []
    boxes = []
#     boxes2 = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            classId = np.argmax(scores)
            confidence = scores[classId]
            if confidence > confThreshold:
                center_x = int(detection[0] * frameWidth)
                center_y = int(detection[1] * frameHeight)
                width = int(detection[2] * frameWidth)
                height = int(detection[3] * frameHeight)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                classIds.append(classId)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
#                 boxes2.append([detection[0], detection[1], detection[2], detection[3]])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)
    f = open(str(currentFrame) + '.txt', 'w')
    for i in indices:
        i = i[0]
        box = boxes[i]
        classId = classIds[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        confidence = confidences[i]
#         box2 = boxes2[i]
        d1 = (left + width / 2) / frameWidth
        d2 = (top + height / 2) / frameHeight
        d3 = width / frameWidth
        d4 = height / frameHeight
#         drawPred(classIds[i], confidences[i], left, top, left + width, top + height)
#         data = "%d %f %f %f %f\n" % (classId, d1, d2, d3, d4)
        if classId == 2 :
            data = "%d %f %f %f %f %f\n" % (0, confidence, d1, d2, d3, d4)
        elif classId == 7 :
            data = "%d %f %f %f %f %f\n" % (1, confidence, d1, d2, d3, d4)
        else :
            pass
        f.write(data)
#     cv2.imwrite(str(currentFrame) + '.jpg', frame)
    currentFrame += 1
    f.close()
cap.release()

프레임을 수신할 수 없습니다(스트림 끝?). 종료 중 ...
